In [7]:
import os
import subprocess
import random
import itertools

igome_project_path = '/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling/'

In [8]:
def get_peptides(file):
    # e.g., b' 10424980 /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling/exp12/
    # first_phase_output/17b_01/TGCTA_Exp12.17b_01..faa\n'
    # return int(subprocess.check_output(f'wc -l {file}', shell=True).decode().split()[0])
    peptides = []
    with open(file) as f:
        for line in f:
            if line.startswith('>'):
                peptides.append(f'{line}{f.readline()}')
    return peptides


# Sample training data

In [11]:
filtered_peptides_path = f'{igome_project_path}/analysis/reads_filtration/'
sampling_ratio = 0.01
m = 5_735_901  # the average number of reads in all exp12 samples
sampling_amount = int(m*sampling_ratio)
print(sampling_amount)
m_noise = m-sampling_amount
biological_conditions = ['17b', 'b12', '21c', 'Herceptin']

hcv_peptides = get_peptides(f'{igome_project_path}/diluted_samples/7295_Exp22.RO4_c.MistakeAllowed.1.AA.fs')

for bc in biological_conditions:
    paths_to_faa = []
    for path, dirs, files in os.walk(filtered_peptides_path):
        for file in files:
            if '05' in file:
                print('*'*30+'Skipping sample number 5 !!!')
            if bc in file and file.endswith('faa'):
                paths_to_faa.append(os.path.join(path, file))
                break

    paths_to_faa.sort(key=lambda x:x.split('_unique')[0])
    peptides_lists = [get_peptides(path) for path in paths_to_faa]
    files_length = [len(peptides_list) for peptides_list in peptides_lists]
    print('\n'.join(paths_to_faa))
    print(files_length)

    all_sample_peptides = []
    for peptides_list in peptides_lists:
        all_sample_peptides.extend(peptides_list)

    for i in range(1,5):
        sampeled_peptides_path = f'{igome_project_path}/diluted_samples/training/{sampling_ratio}/{bc}/sample_{i}/{bc}_{sampling_ratio}.faa'
        print(sampeled_peptides_path)
        if not os.path.exists(os.path.split(sampeled_peptides_path)[0]):
            os.makedirs(os.path.split(sampeled_peptides_path)[0])

        random_indexes = random.choices(range(len(all_sample_peptides)), k=sampling_amount)
        
        print(f'max: {max(random_indexes)}')
        print(f'min: {min(random_indexes)}')
        assert max(random_indexes)<len(all_sample_peptides)
        assert min(random_indexes)>-1

        with open(sampeled_peptides_path, 'w') as f:
            for i in random_indexes:
                f.write(f'{all_sample_peptides[i]}')

        random_indexes = random.choices(range(len(hcv_peptides)), k=m_noise)
        print(f'max: {max(random_indexes)}')
        print(f'min: {min(random_indexes)}')
        assert max(random_indexes)<len(hcv_peptides)
        assert min(random_indexes)>-1

        with open(f'{os.path.split(sampeled_peptides_path)[0]}/HCV_{1-sampling_ratio}.faa', 'w') as f:
            for i in random_indexes:
                f.write(f'{hcv_peptides[i]}')


57359
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//analysis/reads_filtration/17b_01/17b_01_unique_rpm.faa
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//analysis/reads_filtration/17b_02/17b_02_unique_rpm.faa
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//analysis/reads_filtration/17b_03/17b_03_unique_rpm.faa
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//analysis/reads_filtration/17b_04/17b_04_unique_rpm.faa
[30031, 60212, 132394, 41186]
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/17b/sample_1/17b_0.01.faa
max: 263813
min: 8
max: 2129084
min: 0
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/17b/sample_2/17b_0.01.faa
max: 263809
min: 5
max: 2129084
min: 0
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/17b/sample_3/17b_0.01.faa
max: 263822
min: 2
max: 2129084
min: 0
/Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.

# Merge training files

In [12]:
for path,dirs,files in os.walk(f'{igome_project_path}/diluted_samples/training/'):
    if dirs == []:
        for file in files:
            if file.startswith('.'):
                continue
            print(file)
            with open(os.path.join(path, file)) as f:
                text = f.read()
            percent, mAb = path.split('/')[-3:-1]
            with open(os.path.join(path, f'{mAb}_{percent}_HCV_{1-float(percent)}_merged.faa'), 'a') as f:
                f.write(text)
        for file in files:
            if 'merged' not in file:
                print(f'Deleting: {os.path.join(path, file)}')
                os.remove(os.path.join(path, file))

HCV_0.99.faa
Herceptin_0.01.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_2/HCV_0.99.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_2/Herceptin_0.01.faa
HCV_0.99.faa
Herceptin_0.01.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_3/HCV_0.99.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_3/Herceptin_0.01.faa
HCV_0.99.faa
Herceptin_0.01.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_4/HCV_0.99.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herceptin/sample_4/Herceptin_0.01.faa
HCV_0.99.faa
Herceptin_0.01.faa
Deleting: /Users/Oren/Dropbox/Projects/gershoni/IgOmeProfiling//diluted_samples/training/0.01/Herc

# Sample TESTING data


In [13]:
##########
filtered_peptides_path = f'{igome_project_path}/analysis/reads_filtration/'
biological_conditions = ['17b', 'b12', '21c', 'Herceptin']
hcv_peptides = get_peptides(f'{igome_project_path}/diluted_samples/7295_Exp22.RO4_c.MistakeAllowed.1.AA.fs')
m = 5_735_901


for sampling_ratio in [0.01, 0.25]:
    sampling_amount = int(m*sampling_ratio)
    m_noise = m-sampling_amount*2

    import itertools

    for bcs in itertools.combinations(biological_conditions, 2):
        for bc in bcs:
            sampeled_peptides_path = f'{igome_project_path}/diluted_samples/testing/{sampling_ratio}/{bcs[0]}_{sampling_ratio}_{bcs[1]}_{sampling_ratio}_HCV_{1-2*sampling_ratio}/{bc}_{sampling_ratio}.faa'
            if not os.path.exists(os.path.split(sampeled_peptides_path)[0]):
                os.makedirs(os.path.split(sampeled_peptides_path)[0])

            # find peptides file and generate peptides list
            for path, dirs, files in os.walk(filtered_peptides_path):
                for file in files:
                    if bc in file and file.endswith('05..faa'):
                        test_sample_peptides = get_peptides(os.path.join(path, file))
                        break

            # sampling and saving to file
            random_indexes = random.choices(range(len(test_sample_peptides)), k=sampling_amount)
            print(f'max: {max(random_indexes)}')
            print(f'min: {min(random_indexes)}')
            with open(sampeled_peptides_path, 'w') as f:
                for i in random_indexes:
                    f.write(f'{test_sample_peptides[i]}')

        random_indexes = random.choices(range(len(hcv_peptides)), k=m_noise)
        print(f'max: {max(random_indexes)}')
        print(f'min: {min(random_indexes)}')
        assert max(random_indexes)<len(hcv_peptides)
        assert min(random_indexes)>-1

        if not os.path.exists(os.path.split(sampeled_peptides_path)[0]):
            os.makedirs(os.path.split(sampeled_peptides_path)[0])
        with open(f'{igome_project_path}/diluted_samples/testing/{sampling_ratio}/{bcs[0]}_{sampling_ratio}_{bcs[1]}_{sampling_ratio}_HCV_{1-2*sampling_ratio}/HCV_{1-2*sampling_ratio}.faa', 'w') as f:
            for i in random_indexes:
                f.write(f'{hcv_peptides[i]}')


NameError: name 'test_sample_peptides' is not defined

## Merge TESTING files

In [ ]:
for path,dirs,files in os.walk(f'{igome_project_path}/diluted_samples/testing/'):
    if dirs == []:
        for file in files:
            if file.startswith('.'):
                continue
            print(file)
            with open(os.path.join(path, file)) as f:
                text = f.read()
            dir_name = path.split('/')[-1]
            with open(os.path.join(path, f'{dir_name}_merged.faa'), 'a') as f:
                f.write(text)   
        for file in files:
            if 'merged' not in file:
                print(f'Deleting: {os.path.join(path, file)}')
                os.remove(os.path.join(path, file))